In [1]:
import pandas as pd
import requests
import json
from urllib.parse import urlencode
from pytezos.michelson.micheline import blind_unpack
from pprint import pprint
import asyncio


class CoinbaseAPI:
    def __init__(self, uri='https://api.pro.coinbase.com'):
        self.uri = uri

    def get_history_prices(self, pair, start=None, stop=None, granularity=900):
        """ requests history data from coinbase, creates dataframe and makes
            all processing and type transformations """

        params = {
            'granularity': granularity
        }

        if start:
            params['start'] = start

        if stop:
            params['stop'] = stop

        method = f'{self.uri}/products/{pair}/candles?{urlencode(params)}'
        response = requests.get(method)
        headers = ['time', 'low', 'high', 'open', 'close', 'volume']
        df = pd.DataFrame(json.loads(response.text), columns=headers)

        assert not df.time.duplicated().any()

        return df.set_index('time')


class CoinbaseData:

    def __init__(self):
        self.responses_history = []
        self.api = CoinbaseAPI()
        self.update()


    def update(self):
        self.last_prices = self.api.get_history_prices(pair='XTZ-USD', granularity=60)

        # looks like history data in API can be changed,
        # to check this I am storing prev requested data:
        self.responses_history.append(self.last_prices)
        self.responses_history = self.responses_history[-100:]

        # Kolibri data using integers with 10**6 precision:
        self.last_prices_int = (self.last_prices * 1_000_000).astype(int)


    def has_close_timestamp(self, timestamp):
        return timestamp in self.last_prices_int.index


    def check_candle(self, candle, check_columns=['low', 'high', 'open', 'close', 'volume']):
        """ Checks that provided candle is the same as in self.last_prices """

        timestamp = candle.close_timestamp
        true_candle = self.last_prices_int.loc[ timestamp ][ check_columns ]
        candle = candle[ check_columns ]

        is_same_values = all(true_candle == candle)

        if not is_same_values:
            print(f'{timestamp}: data from kolibri differs from coinbase:')
            print('kolibri_candle: \n', candle)
            print('coinbase_candle: \n', true_candle)
            return False
        else:
            print(f'{timestamp}: data in kolibri is the same as in coinbase')
            return True


    async def autoupdate(self, period=30):
        while True:
            self.update()
            await asyncio.sleep(period)


    def get_timestamp_request_history(self, timestamp):
        """ Returns all history data for given timestamp """

        return pd.DataFrame([
            df.assign(last_index=max(df.index)).loc[timestamp]
            for df in self.responses_history
            if timestamp in df.index
        ])


class WrongDataException(Exception):
    pass


def unpack_xtz_price(oracle_data):
    micheline_bytes = bytes.fromhex(oracle_data['messages'][2])
    michelson = blind_unpack(micheline_bytes)
    if michelson[0] != 'XTZ-USD':
        raise WrongDataException(f'michelson contains: {michelson} instead of canlde data')

    return pd.Series({
        'open_timestamp': michelson[1][0],
        'close_timestamp': michelson[1][1][0],
        'open': michelson[1][1][1][0],
        'high': michelson[1][1][1][1][0],
        'low': michelson[1][1][1][1][1][0],
        'close': michelson[1][1][1][1][1][1][0],
        'volume': michelson[1][1][1][1][1][1][1]
    })


def get_last_kolibri_data():
    response = requests.get('https://oracle-data.kolibri.finance/data.json')
    kolibri_raw = json.loads(response.text)
    return unpack_xtz_price(kolibri_raw)


async def check_candle_while_succeed_task(coinbase_data, candle, attempts=10):
    for _ in range(attempts):
        if coinbase_data.has_close_timestamp(candle.close_timestamp):
            return coinbase_data.check_candle(candle)
        await asyncio.sleep(30)


async def check_data_in_cycle(coinbase_data):
    last_close_timestamp = 0
    # coinbase_data = CoinbaseData()
    autoupdate_task = asyncio.create_task(coinbase_data.autoupdate())
    stats = {
        'same': 0,
        'different': 0
    }

    while True:
        try:
            kolibri_candle = get_last_kolibri_data()

        except WrongDataException as e:
            """ This is strange exception when data in kolibri contains
                signature instead of candle data """

            # print(f'wrong data in kolibri oracle, {str(e)}')
            await asyncio.sleep(10)
            continue

        is_new_data = kolibri_candle['close_timestamp'] > last_close_timestamp

        if is_new_data:
            is_same = await check_candle_while_succeed_task(coinbase_data, kolibri_candle)

            if is_same:
                stats['same'] += 1
            else:
                stats['different'] += 1

            print(f"stats --- same: {stats['same']}, different: {stats['different']}")

        last_close_timestamp = kolibri_candle['close_timestamp']

        await asyncio.sleep(30)

    await autoupdate_task

### Running data checker:

In [ ]:
coinbase_data = CoinbaseData()
await check_data_in_cycle(coinbase_data)

1626104580: data in kolibri is the same as in coinbase
stats --- same: 1, different: 0
1626104640: data in kolibri is the same as in coinbase
stats --- same: 2, different: 0
1626104700: data from kolibri differs from coinbase:
kolibri_candle: 
 low          2842100
high         2846800
open         2843200
close        2846800
volume    1743840000
dtype: int64
coinbase_candle: 
 low       2843200
high      2843200
open      2843200
close     2843200
volume    4550000
Name: 1626104700, dtype: int64
stats --- same: 2, different: 1
1626104880: data in kolibri is the same as in coinbase
stats --- same: 3, different: 1


### Check coinbase API requests history for given timestamp:

In [10]:
df = coinbase_data.get_timestamp_request_history(1626103440)
(df == df.iloc[0]).all()

low           False
high           True
open           True
close         False
volume        False
last_index    False
dtype: bool

In [11]:
df

,low,high,open,close,volume,last_index
1626103440,2.8624,2.8642,2.8642,2.8624,50.89,1.626103e+09
1626103440,2.8624,2.8642,2.8642,2.8624,50.89,1.626103e+09
1626103440,2.8624,2.8642,2.8642,2.8624,50.89,1.626103e+09
1626103440,2.8624,2.8642,2.8642,2.8624,50.89,1.626103e+09
1626103440,2.8624,2.8642,2.8642,2.8624,50.89,1.626103e+09
1626103440,2.8624,2.8642,2.8642,2.8624,50.89,1.626103e+09
1626103440,2.8624,2.8642,2.8642,2.8624,50.89,1.626103e+09
1626103440,2.8624,2.8642,2.8642,2.8624,50.89,1.626103e+09
1626103440,2.8624,2.8642,2.8642,2.8624,50.89,1.626103e+09
1626103440,2.8624,2.8642,2.8642,2.8624,50.89,1.626103e+09
